In [ ]:
"""
猫狗分类模型 - 基础卷积神经网络实现

本notebook演示如何从头构建一个卷积神经网络(CNN)来进行二分类任务。
通过这个示例,你将学习到:
1. 卷积神经网络的基本架构设计
2. 数据预处理和归一化的重要性
3. 模型编译和训练的完整流程
4. 过拟合现象的识别和分析

技术要点:
- 使用多层卷积-池化结构提取图像特征
- 采用Sigmoid激活函数实现二分类
- RMSprop优化器适合处理非平稳目标

作者: [Your Name]
日期: 2024-01
"""

import os
import warnings
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers

# 禁用不必要的警告信息
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 设置随机种子以确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
"""
数据路径配置模块

使用相对路径确保代码的可移植性。
数据集目录结构应该如下:
猫狗数据集/
├── dataset/
│   ├── train/
│   │   ├── cats/
│   │   └── dogs/
│   ├── validation/
│   │   ├── cats/
│   │   └── dogs/
│   └── test/
│       ├── cats/
│       └── dogs/
"""

# 数据集根目录配置(相对于notebook的路径)
DATA_ROOT = Path("猫狗数据集/dataset")

# 各个数据集的子目录
TRAIN_DIR = DATA_ROOT / "train"
VALIDATION_DIR = DATA_ROOT / "validation"
TEST_DIR = DATA_ROOT / "test"

# 验证路径是否存在
print("=" * 60)
print("数据路径验证")
print("=" * 60)
for name, path in [("训练集", TRAIN_DIR), 
                    ("验证集", VALIDATION_DIR), 
                    ("测试集", TEST_DIR)]:
    exists = path.exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path} {'(存在)' if exists else '(不存在)'}")
    
    if exists:
        # 统计每个类别的样本数量
        cats_count = len(list((path / "cats").glob("*.jpg"))) if (path / "cats").exists() else 0
        dogs_count = len(list((path / "dogs").glob("*.jpg"))) if (path / "dogs").exists() else 0
        print(f"   猫: {cats_count}张, 狗: {dogs_count}张, 总计: {cats_count + dogs_count}张")

print("=" * 60)

In [ ]:
"""
模型超参数配置

这些参数控制模型的训练行为和网络结构。
在生产环境中，通常会通过配置文件或命令行参数来管理这些设置。
"""

# 图像预处理参数
IMG_HEIGHT = 150  # 输入图像高度
IMG_WIDTH = 150   # 输入图像宽度
IMG_CHANNELS = 3  # RGB图像通道数

# 训练参数
BATCH_SIZE = 32        # 批次大小,影响训练速度和内存占用
EPOCHS = 2             # 训练轮数(测试用,生产环境建议30+)
LEARNING_RATE = 1e-4   # 学习率,控制参数更新步长

# 打印配置信息
print("=" * 60)
print("模型配置")
print("=" * 60)
print(f"输入图像尺寸: {IMG_HEIGHT}x{IMG_WIDTH}x{IMG_CHANNELS}")
print(f"批次大小: {BATCH_SIZE}")
print(f"训练轮数: {EPOCHS} (注意: 当前为测试配置)")
print(f"学习率: {LEARNING_RATE}")
print("=" * 60)

In [ ]:
"""
构建卷积神经网络模型

网络架构设计说明:
1. 4个卷积块,每个块包含:
   - Conv2D: 卷积层,提取图像特征
   - MaxPooling2D: 池化层,降低特征维度,增强特征鲁棒性
   
2. 特征图数量递增(32→64→128→128):
   - 浅层捕获边缘、纹理等低级特征
   - 深层捕获物体部件、形状等高级特征
   
3. 全连接层:
   - Flatten: 将2D特征图展平为1D向量
   - Dense(512): 特征整合和高级抽象
   - Dense(1): 二分类输出(sigmoid激活)

为什么使用这种架构?
- 多层卷积逐步提取抽象特征
- 池化层减少计算量并增强平移不变性
- 特征图递增平衡了计算效率和表达能力
"""

def build_cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    """
    构建用于二分类的卷积神经网络
    
    Args:
        input_shape: 输入图像的形状 (height, width, channels)
        
    Returns:
        未编译的Keras模型
    """
    model = models.Sequential(name='Cat_Dog_CNN')
    
    # 第1个卷积块: 32个3x3卷积核
    model.add(layers.Conv2D(
        32, (3, 3), 
        activation='relu',
        input_shape=input_shape,
        name='conv1'
    ))
    model.add(layers.MaxPooling2D((2, 2), name='pool1'))
    
    # 第2个卷积块: 64个3x3卷积核
    model.add(layers.Conv2D(64, (3, 3), activation='relu', name='conv2'))
    model.add(layers.MaxPooling2D((2, 2), name='pool2'))
    
    # 第3个卷积块: 128个3x3卷积核
    model.add(layers.Conv2D(128, (3, 3), activation='relu', name='conv3'))
    model.add(layers.MaxPooling2D((2, 2), name='pool3'))
    
    # 第4个卷积块: 128个3x3卷积核
    model.add(layers.Conv2D(128, (3, 3), activation='relu', name='conv4'))
    model.add(layers.MaxPooling2D((2, 2), name='pool4'))
    
    # 展平特征图
    model.add(layers.Flatten(name='flatten'))
    
    # 全连接层
    model.add(layers.Dense(512, activation='relu', name='fc1'))
    
    # 输出层: 二分类使用sigmoid激活
    model.add(layers.Dense(1, activation='sigmoid', name='output'))
    
    return model

# 创建模型实例
model = build_cnn_model()

# 显示模型结构
print("\n" + "=" * 60)
print("模型架构")
print("=" * 60)
model.summary()
print("=" * 60)

# 计算模型参数量
total_params = model.count_params()
print(f"\n总参数量: {total_params:,} ({total_params/1e6:.2f}M)")

In [ ]:
"""
数据加载和预处理

使用ImageDataGenerator进行数据预处理:
1. rescale=1./255: 将像素值从[0,255]归一化到[0,1]
   - 神经网络在小范围数值上训练更稳定
   - 避免梯度爆炸/消失问题
   
2. flow_from_directory: 从目录结构自动加载数据
   - 自动识别子文件夹作为类别标签
   - 批量加载,节省内存
   
注意: 此阶段不使用数据增强,仅做归一化
"""

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 创建数据生成器(仅归一化)
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 训练集生成器
train_generator = train_datagen.flow_from_directory(
    str(TRAIN_DIR),  # 转换Path对象为字符串
    target_size=(IMG_HEIGHT, IMG_WIDTH),  # 调整所有图像到统一尺寸
    batch_size=BATCH_SIZE,
    class_mode='binary',  # 二分类模式,标签为0或1
    shuffle=True,         # 每个epoch打乱数据
    seed=RANDOM_SEED      # 设置随机种子保证可复现
)

# 验证集生成器
validation_generator = validation_datagen.flow_from_directory(
    str(VALIDATION_DIR),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # 验证集不需要打乱
)

# 测试集生成器
test_generator = test_datagen.flow_from_directory(
    str(TEST_DIR),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# 显示数据加载信息
print("\n" + "=" * 60)
print("数据生成器信息")
print("=" * 60)
print(f"类别映射: {train_generator.class_indices}")
print(f"训练集批次数: {len(train_generator)}")
print(f"验证集批次数: {len(validation_generator)}")
print(f"测试集批次数: {len(test_generator)}")

# 查看一个批次的数据形状
sample_batch, sample_labels = next(iter(train_generator))
print(f"\n单批次数据形状: {sample_batch.shape}")
print(f"单批次标签形状: {sample_labels.shape}")
print(f"像素值范围: [{sample_batch.min():.3f}, {sample_batch.max():.3f}]")
print("=" * 60)

In [ ]:
"""
模型编译和训练

编译配置说明:
1. optimizer=RMSprop: 
   - 适合处理非平稳目标和RNN
   - 使用移动平均来平滑梯度
   
2. loss=binary_crossentropy:
   - 二分类的标准损失函数
   - 衡量预测概率分布与真实分布的差异
   
3. metrics=['accuracy']:
   - 监控分类准确率
   - 注意: accuracy仅用于监控,不参与优化
   
训练策略:
- 计算steps_per_epoch确保遍历完整数据集
- 使用validation_data监控泛化能力
- 当前epochs=2仅用于快速测试
"""

# 编译模型
model.compile(
    optimizer=optimizers.RMSprop(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n" + "=" * 60)
print("开始训练模型")
print("=" * 60)
print(f"训练样本数: {train_generator.samples}")
print(f"验证样本数: {validation_generator.samples}")
print(f"批次大小: {BATCH_SIZE}")
print(f"训练轮数: {EPOCHS}")
print("=" * 60 + "\n")

# 计算每个epoch的步数
# steps_per_epoch = 样本总数 // 批次大小
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = validation_generator.samples // BATCH_SIZE

# 训练模型
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    verbose=1  # 显示进度条
)

print("\n" + "=" * 60)
print("训练完成")
print("=" * 60)

In [ ]:
"""
查看训练历史记录

history.history字典包含每个epoch的指标值:
- loss: 训练集损失
- accuracy: 训练集准确率
- val_loss: 验证集损失
- val_accuracy: 验证集准确率

通过对比训练集和验证集的指标,可以判断模型是否过拟合:
- 如果训练准确率持续上升,但验证准确率停滞或下降 → 过拟合
- 如果两者都持续上升 → 模型正常学习
"""

print("=" * 60)
print("训练历史记录的指标")
print("=" * 60)
print(f"可用指标: {list(history.history.keys())}")
print("=" * 60)

# 显示最终的训练结果
final_train_loss = history.history['loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_loss = history.history['val_loss'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"\n最终训练结果 (Epoch {EPOCHS}):")
print(f"  训练损失: {final_train_loss:.4f}")
print(f"  训练准确率: {final_train_acc:.4f}")
print(f"  验证损失: {final_val_loss:.4f}")
print(f"  验证准确率: {final_val_acc:.4f}")
print("=" * 60)

In [ ]:
"""
可视化训练过程

通过绘制训练和验证的损失/准确率曲线,可以直观地:
1. 观察模型收敛情况
2. 识别过拟合现象
3. 判断是否需要更多训练轮数

过拟合的典型特征:
- 训练损失持续下降,验证损失上升或波动
- 训练准确率持续上升,验证准确率停滞或下降
- 两条曲线之间的差距越来越大
"""

import matplotlib.pyplot as plt

# 提取训练历史数据
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(acc) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 绘制准确率曲线
ax1.plot(epochs_range, acc, 'bo-', label='训练准确率', linewidth=2, markersize=8)
ax1.plot(epochs_range, val_acc, 'rs-', label='验证准确率', linewidth=2, markersize=8)
ax1.set_title('训练和验证准确率', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('准确率', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 绘制损失曲线
ax2.plot(epochs_range, loss, 'bo-', label='训练损失', linewidth=2, markersize=8)
ax2.plot(epochs_range, val_loss, 'rs-', label='验证损失', linewidth=2, markersize=8)
ax2.set_title('训练和验证损失', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('损失', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 分析过拟合程度
if len(acc) > 1:
    acc_gap = acc[-1] - val_acc[-1]
    print("\n" + "=" * 60)
    print("过拟合分析")
    print("=" * 60)
    print(f"训练准确率: {acc[-1]:.4f}")
    print(f"验证准确率: {val_acc[-1]:.4f}")
    print(f"准确率差距: {acc_gap:.4f}")
    
    if acc_gap > 0.1:
        print("\n⚠️  检测到明显过拟合!")
        print("建议:")
        print("  1. 使用数据增强")
        print("  2. 添加Dropout层")
        print("  3. 减少模型复杂度")
        print("  4. 使用L2正则化")
    elif acc_gap > 0.05:
        print("\n⚠️  检测到轻微过拟合")
        print("可以考虑使用正则化技术")
    else:
        print("\n✓ 模型泛化良好")
    print("=" * 60)

In [ ]:
"""
数据增强技术

数据增强(Data Augmentation)是提高模型泛化能力的重要技术。
通过对训练图像应用随机变换,可以:
1. 增加训练数据的多样性
2. 减少过拟合
3. 提高模型对图像变化的鲁棒性

各参数说明:
- rotation_range=40: 随机旋转±40度
- width_shift_range=0.2: 水平平移±20%图像宽度
- height_shift_range=0.2: 垂直平移±20%图像高度
- shear_range=0.2: 剪切变换强度
- zoom_range=0.2: 随机缩放±20%
- horizontal_flip=True: 随机水平翻转
- fill_mode='nearest': 填充新像素的方法

注意: 数据增强只应用于训练集,不应用于验证集和测试集!
"""

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 创建带数据增强的生成器
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,              # 归一化
    rotation_range=40,           # 随机旋转
    width_shift_range=0.2,       # 水平平移
    height_shift_range=0.2,      # 垂直平移
    shear_range=0.2,             # 剪切变换
    zoom_range=0.2,              # 随机缩放
    horizontal_flip=True,        # 水平翻转
    fill_mode='nearest'          # 填充策略
)

print("=" * 60)
print("数据增强配置")
print("=" * 60)
print("已启用的增强技术:")
print("  ✓ 随机旋转 (±40°)")
print("  ✓ 随机水平/垂直平移 (±20%)")
print("  ✓ 剪切变换 (强度: 0.2)")
print("  ✓ 随机缩放 (±20%)")
print("  ✓ 随机水平翻转")
print("\n注意: 增强仅应用于训练集")
print("=" * 60)

In [ ]:
"""
可视化数据增强效果

通过展示同一张图片的多个增强版本,可以直观理解数据增强的作用。
这有助于:
1. 验证增强参数是否合理
2. 检查是否有过度增强(导致图像失真)
3. 确保增强后的图像仍然可识别
"""

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import matplotlib.pyplot as plt
import numpy as np

# 选择一张样本图片
# 尝试从训练集中找一张猫的图片
sample_img_path = None
cats_dir = TRAIN_DIR / "cats"
if cats_dir.exists():
    cat_images = list(cats_dir.glob("*.jpg"))
    if cat_images:
        sample_img_path = cat_images[0]

if sample_img_path is None:
    print("警告: 未找到样本图片,跳过可视化")
else:
    # 加载图片
    img = load_img(sample_img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)  # 转换为(1, height, width, 3)
    
    # 生成4个增强版本
    print(f"原始图片: {sample_img_path.name}")
    print("生成增强样本...")
    
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.ravel()
    
    i = 0
    for batch in train_datagen_augmented.flow(x, batch_size=1):
        axes[i].imshow(array_to_img(batch[0]))
        axes[i].set_title(f'增强样本 {i+1}', fontsize=12, fontweight='bold')
        axes[i].axis('off')
        i += 1
        if i >= 4:
            break
    
    plt.suptitle('数据增强效果展示', fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    print("=" * 60)
    print("数据增强效果说明")
    print("=" * 60)
    print("观察点:")
    print("  - 图像是否保持可识别性")
    print("  - 变换是否自然合理")
    print("  - 是否有明显失真")
    print("=" * 60)

In [ ]:
"""
改进模型: 添加Dropout正则化

Dropout是一种强大的正则化技术:
1. 原理: 训练时随机"丢弃"部分神经元
2. 效果: 防止神经元过度依赖特定连接
3. 结果: 提高模型泛化能力,减少过拟合

Dropout rate=0.5的含义:
- 训练时: 每个神经元有50%概率被临时移除
- 测试时: 使用所有神经元,但权重按比例缩放

为什么放在Flatten之后?
- 卷积层通常不加Dropout(或使用SpatialDropout2D)
- 全连接层更容易过拟合,需要Dropout保护
"""

def build_cnn_with_dropout(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), 
                           dropout_rate=0.5):
    """
    构建带Dropout的卷积神经网络
    
    Args:
        input_shape: 输入图像形状
        dropout_rate: Dropout比例,范围[0,1]
        
    Returns:
        未编译的Keras模型
    """
    model = models.Sequential(name='Cat_Dog_CNN_Dropout')
    
    # 卷积块 1-4 (与原模型相同)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                            input_shape=input_shape, name='conv1'))
    model.add(layers.MaxPooling2D((2, 2), name='pool1'))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu', name='conv2'))
    model.add(layers.MaxPooling2D((2, 2), name='pool2'))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu', name='conv3'))
    model.add(layers.MaxPooling2D((2, 2), name='pool3'))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu', name='conv4'))
    model.add(layers.MaxPooling2D((2, 2), name='pool4'))
    
    # 展平
    model.add(layers.Flatten(name='flatten'))
    
    # 添加Dropout (关键改进!)
    model.add(layers.Dropout(dropout_rate, name='dropout'))
    
    # 全连接层
    model.add(layers.Dense(512, activation='relu', name='fc1'))
    
    # 输出层
    model.add(layers.Dense(1, activation='sigmoid', name='output'))
    
    return model

# 创建改进后的模型
model_with_dropout = build_cnn_with_dropout(dropout_rate=0.5)

print("=" * 60)
print("改进模型架构 (添加Dropout)")
print("=" * 60)
model_with_dropout.summary()
print("=" * 60)
print("\n关键改进:")
print("  ✓ 在全连接层前添加Dropout(rate=0.5)")
print("  ✓ 预期效果: 显著减少过拟合")
print("=" * 60)

In [ ]:
"""
使用数据增强和Dropout训练改进模型

综合应用两种正则化技术:
1. 数据增强: 扩充训练数据多样性
2. Dropout: 防止神经网络过拟合

训练策略:
- 使用增强后的训练数据
- 验证集不做增强(评估真实泛化能力)
- 较小的学习率确保稳定收敛
"""

# 编译改进后的模型
model_with_dropout.compile(
    optimizer=optimizers.RMSprop(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 创建增强训练集生成器
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    str(TRAIN_DIR),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=RANDOM_SEED
)

# 计算训练步数
steps_per_epoch_aug = train_generator_augmented.samples // BATCH_SIZE
validation_steps_aug = validation_generator.samples // BATCH_SIZE

print("\n" + "=" * 60)
print("开始训练改进模型 (数据增强 + Dropout)")
print("=" * 60)
print(f"训练样本数: {train_generator_augmented.samples}")
print(f"验证样本数: {validation_generator.samples}")
print(f"数据增强: 已启用")
print(f"Dropout率: 0.5")
print(f"训练轮数: {EPOCHS}")
print("=" * 60 + "\n")

# 训练模型
history_augmented = model_with_dropout.fit(
    train_generator_augmented,
    steps_per_epoch=steps_per_epoch_aug,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_steps_aug,
    verbose=1
)

print("\n" + "=" * 60)
print("改进模型训练完成")
print("=" * 60)

# 显示最终结果
final_train_loss_aug = history_augmented.history['loss'][-1]
final_train_acc_aug = history_augmented.history['accuracy'][-1]
final_val_loss_aug = history_augmented.history['val_loss'][-1]
final_val_acc_aug = history_augmented.history['val_accuracy'][-1]

print(f"\n最终训练结果 (Epoch {EPOCHS}):")
print(f"  训练损失: {final_train_loss_aug:.4f}")
print(f"  训练准确率: {final_train_acc_aug:.4f}")
print(f"  验证损失: {final_val_loss_aug:.4f}")
print(f"  验证准确率: {final_val_acc_aug:.4f}")

# 对比原始模型
print(f"\n与原始模型对比:")
print(f"  准确率差距(原始): {acc[-1] - val_acc[-1]:.4f}")
print(f"  准确率差距(改进): {final_train_acc_aug - final_val_acc_aug:.4f}")
print("=" * 60)

In [ ]:
"""
保存训练好的模型

模型保存的重要性:
1. 避免重复训练,节省时间和计算资源
2. 部署模型到生产环境
3. 后续进行模型评估和分析

Keras支持两种保存格式:
- .h5格式 (HDF5): 传统格式,兼容性好
- SavedModel格式: TensorFlow推荐,支持更多特性
"""

# 定义模型保存路径
MODEL_SAVE_DIR = Path("models")
MODEL_SAVE_DIR.mkdir(exist_ok=True)

model_save_path = MODEL_SAVE_DIR / "cat_dog_classifier_improved.h5"

# 保存模型
try:
    model_with_dropout.save(str(model_save_path))
    print("=" * 60)
    print("模型保存")
    print("=" * 60)
    print(f"✓ 模型已保存到: {model_save_path}")
    print(f"  文件大小: {model_save_path.stat().st_size / (1024*1024):.2f} MB")
    print("=" * 60)
    
    # 验证模型可以加载
    print("\n验证模型加载...")
    loaded_model = models.load_model(str(model_save_path))
    print("✓ 模型加载成功")
    
except Exception as e:
    print(f"✗ 模型保存失败: {e}")

In [ ]:
"""
检查改进模型的训练历史

查看history对象中保存的指标,用于后续可视化和分析
"""

print("=" * 60)
print("改进模型训练历史指标")
print("=" * 60)
print(f"可用指标: {list(history_augmented.history.keys())}")
print(f"训练轮数: {len(history_augmented.history['loss'])}")
print("=" * 60)

In [ ]:
"""
可视化改进模型的训练过程

对比原始模型和改进模型的表现,验证正则化技术的效果
"""

import matplotlib.pyplot as plt

# 提取改进模型的训练历史
acc_aug = history_augmented.history['accuracy']
val_acc_aug = history_augmented.history['val_accuracy']
loss_aug = history_augmented.history['loss']
val_loss_aug = history_augmented.history['val_loss']
epochs_range_aug = range(1, len(acc_aug) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 绘制准确率曲线
ax1.plot(epochs_range_aug, acc_aug, 'bo-', label='训练准确率', linewidth=2, markersize=8)
ax1.plot(epochs_range_aug, val_acc_aug, 'rs-', label='验证准确率', linewidth=2, markersize=8)
ax1.set_title('训练和验证准确率 (数据增强+Dropout)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('准确率', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 绘制损失曲线
ax2.plot(epochs_range_aug, loss_aug, 'bo-', label='训练损失', linewidth=2, markersize=8)
ax2.plot(epochs_range_aug, val_loss_aug, 'rs-', label='验证损失', linewidth=2, markersize=8)
ax2.set_title('训练和验证损失 (数据增强+Dropout)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('损失', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 分析改进效果
if len(acc_aug) > 1:
    acc_gap_aug = acc_aug[-1] - val_acc_aug[-1]
    print("\n" + "=" * 60)
    print("改进模型 - 过拟合分析")
    print("=" * 60)
    print(f"训练准确率: {acc_aug[-1]:.4f}")
    print(f"验证准确率: {val_acc_aug[-1]:.4f}")
    print(f"准确率差距: {acc_gap_aug:.4f}")
    
    if acc_gap_aug > 0.1:
        print("\n⚠️  仍存在过拟合,可进一步:")
        print("  - 增加训练轮数")
        print("  - 调整Dropout率")
        print("  - 尝试更强的数据增强")
    elif acc_gap_aug > 0.05:
        print("\n✓ 过拟合已显著改善")
    else:
        print("\n✓ 模型泛化优秀!")
    print("=" * 60)

# 对比原始模型和改进模型
print("\n" + "=" * 60)
print("模型对比总结")
print("=" * 60)
print(f"原始模型 - 训练准确率: {acc[-1]:.4f}, 验证准确率: {val_acc[-1]:.4f}, 差距: {acc[-1]-val_acc[-1]:.4f}")
print(f"改进模型 - 训练准确率: {acc_aug[-1]:.4f}, 验证准确率: {val_acc_aug[-1]:.4f}, 差距: {acc_gap_aug:.4f}")
print(f"\n验证准确率提升: {(val_acc_aug[-1] - val_acc[-1]):.4f}")
print(f"过拟合程度降低: {((acc[-1]-val_acc[-1]) - acc_gap_aug):.4f}")
print("=" * 60)

In [ ]:
"""
总结与后续步骤

本notebook涵盖的内容:
✓ 从头构建CNN进行二分类
✓ 理解过拟合现象及其诊断方法
✓ 应用数据增强技术
✓ 使用Dropout进行正则化
✓ 对比不同技术的效果

后续改进方向:
1. 使用预训练模型(VGG, ResNet等) - 参见其他notebook
2. 模型微调(Fine-tuning)
3. 使用学习率调度(Learning Rate Scheduling)
4. 尝试其他正则化技术(BatchNormalization, L2正则化)
5. 集成学习(Ensemble)

生产部署考虑:
- 模型量化以减小模型大小
- 转换为TensorFlow Lite用于移动设备
- 设置合理的推理阈值
- 实现错误处理和日志记录
"""

print("=" * 60)
print("notebook执行完成")
print("=" * 60)
print("\n关键成果:")
print("  ✓ 基础CNN模型已训练")
print("  ✓ 改进模型(数据增强+Dropout)已训练")
print("  ✓ 模型已保存到本地")
print("\n建议:")
print("  - 在完整数据集上训练更多epochs(30+)")
print("  - 尝试使用预训练模型提升性能")
print("  - 在测试集上评估最终性能")
print("=" * 60)